In [14]:

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import regionmask
import pandas as pd
from geopy.geocoders import Nominatim
from matplotlib.colors import SymLogNorm
from matplotlib.pyplot import cm
import psutil 

import argparse

import xesmf as xe
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import dask
import utils
import gc
import geopandas
import dask.array as da

from numba import jit
import numpy as np
from numba import guvectorize, float64, int64, void

import scipy.signal as signal
import sparse

####### There are three options for the type of run: weighted_co2, annual_co2, and age_retire.
####### weighted_co2 = shutdowns occur based on the percentile of capacity weighted co2 emissions (dirtier plants = bigger emissions)
####### annual_co2 = shutdowns occur based on the percentile of annual co2 emissions (so bigger plants likely = bigger co2 emissions)
####### age_retire = shutdowns occure based on the age of the plant



In [15]:
height = pd.read_excel(f'{utils.raw_data_in_path}gc_72_estimate.xlsx', index_col = 0)
height = height.reindex(index=height.index[::-1])
height_ds = height.diff().dropna().to_xarray().rename({'L':'lev'})
height_ds = height_ds.rename({'Altitude (km)':'dz'}) 
height_ds['dz']*=1e3 #convert to meters
height_ds['dz'].attrs = {'units':'m'}

In [18]:
height2 = pd.read_excel('/net/fs11/d0/emfreese/BC-IRF/to_remove/GC_lev72.xlsx')
height2_ds = (height2.loc[height2['L'].isna()].diff().set_index(np.arange(73,0,-1))['Altitude'].dropna()[::-1].to_xarray().rename({'index':'lev'})*-1e3).sortby('lev')


pressure = pd.read_excel('/net/fs11/d0/emfreese/BC-IRF/to_remove/GC_lev72.xlsx')
pressure_ds = (pressure.loc[~pressure['L'].isna()].set_index(np.arange(72,0,-1))['Pressure']*100).to_xarray().rename({'index':'lev'}).sortby('lev')




In [19]:
height

,Altitude (km),Pressure (hPa)
L,,
0,-0.006,1013.250
1,0.058,1005.650
2,0.189,990.408
3,0.320,975.122
4,0.454,959.837
...,...,...
68,67.243,0.078
69,69.440,0.057
70,71.812,0.040


In [2]:

years = 50

country_emit = 'MALAYSIA'

## Add time dimension in terms of days
length_simulation = years*365
time_array = np.arange(0, length_simulation)

## Days per season
season_days = {'DJF': 90, 'MAM':92, 'JJA':92, 'SON':91}


## import the china global powerplant database from Springer et al.
CGP_df = pd.read_csv('mod_coal_inputs/BC_SE_Asia_all_financing_SEA_GAINS_Springer.csv')

CGP_df.columns = CGP_df.columns.str.replace(' ', '_')

CGP_df = CGP_df.rename(columns = {'YEAR':'Year_of_Commission', 'EMISFACTOR.PLATTS':'CO2_weighted_capacity_1000tonsperMW'})
min_year = CGP_df['Year_of_Commission'].min()
## reduce to one country for emissions

CGP_df = CGP_df.loc[CGP_df['COUNTRY'] == country_emit]
print('Emis data prepped and loaded')


######## Country mask and dataframe ######

country_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_50
country_df = geopandas.read_file('ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')
countries = ['China','Australia', 'India','Myanmar', 'Cambodia', 'Laos','Philippines','Nepal','Bangladesh','Thailand','Bhutan','Brunei','Singapore', 'Papua New Guinea', 'Solomon Islands', 'East Timor', 'Taiwan']
country_df = country_df.rename(columns = {'SOVEREIGNT':'country'})

ds_area = xr.open_dataset('/net/fs11/d0/emfreese/GCrundirs/IRF_runs/stretch_2x_pulse/SEA/Jan/mod_output/GEOSChem.SpeciesConc.20160101_0000z.nc4', engine = 'netcdf4')
utils.fix_area_ij_latlon(ds_area);


regrid_area_ds = xr.open_dataset('Outputs/regridded_population_data.nc')


####### Functions #########

def individual_plant_shutdown(years_running, df, time_array, typical_shutdown_years, unique_id):
    ''' Shutdown a unit early. The df must have a variable 'Year_of_Commission' describing when the plant was comissioned, and 'BC_(g/day)' for BC emissions in g/day
        years_running is the number of years the plant runs
        time_array is the length of time for our simulation
        shutdown_years is the typical lifetime of a coal plant
        unique_id is the unique identifier of a unit'''
    shutdown_days = typical_shutdown_years*365
    E = np.zeros(len(time_array))
    ID_df = df.loc[df['unique_ID'] == unique_id]
    yr_offset = (ID_df['Year_of_Commission'].iloc[0] - min_year)
    test_array = np.where((time_array <= (yr_offset + years_running)*365) & (time_array >= yr_offset * 365), True, False)
    E += test_array* ID_df['BC_(g/day)'].sum()
    return(E)




########## Create emissions profile for each plant over our shutdown times ##########

E_CO2_all_opts = {}
year = 1
for unique_id in CGP_df.loc[CGP_df['BC_(g/day)'] >0]['unique_ID'].values:
    E_CO2_all_opts[unique_id] = individual_plant_shutdown(year, CGP_df, time_array, 40, unique_id)
print('Emissions profiles based on weighted capacity of CO2 emissions percentiles created')





Emis data prepped and loaded
Emissions profiles based on weighted capacity of CO2 emissions percentiles created


In [6]:
############### Convolution, selection of location, health impact assessment ##########
    
## Dictionary for location and time names in the Green's Functions (season and location dependent)
country_emit_dict = {'INDONESIA':['Indo_Jan', 'Indo_Apr', 'Indo_July','Indo_Oct'], 'CAMBODIA':['Cambod_Jan', 'Cambod_Apr', 'Cambod_July','Cambod_Oct'] , 
               'MALAYSIA': ['Malay_Jan','Malay_Apr','Malay_July','Malay_Oct'], 'VIETNAM': ['Viet_Jan','Viet_Apr','Viet_July','Viet_Oct']}

#import the green's function and set our time step
G = xr.open_dataarray('Outputs/G_combined_new.nc', chunks = 'auto')

#column sum Green's function, only select our country of emissions
#G_column_sum = G.where(G.run.isin(country_emit_dict[country_emit]), drop = True).sum(dim = 'lev').compute()
#G_column_sum = G_column_sum.where((G_column_sum > 0), drop = True).rename({'time':'s'})

#select only the surface level for concentration at the surface, only select our country of emissions
G_lev0 = G.where(G.run.isin(country_emit_dict[country_emit]), drop = True).isel(lev = 0).compute()
G_lev0 = G_lev0.where((G_lev0 > 0), drop = True).rename({'time':'s'})

print('G prepped')

G prepped


In [7]:

## function for creating a time specific xarray data array

def np_to_xr_time_specific(C, G, E, time_init):
    '''Function to create an xarray data-array over a specified time period, with lat, lon, and s dimensions (s = time)'''
    C = xr.DataArray(
    data = C,
    dims = ['s','lat','lon'],
    coords = dict(
        s = (['s'], np.arange(time_init, C.shape[0] + time_init)), 
        lat = (['lat'], G.lat.values),
        lon = (['lon'], G.lon.values)
            )
        )
    return(C)

In [28]:



## convolution
yr = 1
for unique_id in CGP_df.loc[CGP_df['BC_(g/day)'] >0]['unique_ID']: #loop over each individual plant we're looking at
    print(unique_id)
    data = pd.DataFrame(columns = ['BC_surface_mean_conc','BC_pop_weight_mean_conc', 'BC_column_mean_conc'], index = countries) #create a dataframe where we output the mean concentration over the year + population weighted mean concentration (all at the surface)
    #concentration
    #####start new#############
    C_conv_sfc = {}
    C_conv_column = {}
    print(E_CO2_all_opts[unique_id])
    if E_CO2_all_opts[unique_id].sum()>0:
        
        yr_range = np.unique([int(i/365) for i, x in enumerate(E_CO2_all_opts[unique_id]>0) if x])[0]
        print(yr_range, 'year range')

        n = np.unique([int(i) for i, x in enumerate(E_CO2_all_opts[unique_id]>0) if x])[0]
            
        for C_dict, G_ds, nm in zip([C_conv_sfc, C_conv_column], [G_lev0, G_column_sum], ['surface', 'column']): 

            for idx, season in enumerate(season_days.keys()):
                C_dict[season] = signal.convolve(G_ds.sel(run = country_emit_dict[country_emit][idx]).fillna(0), 
                             E_CO2_all_opts[unique_id][n:n+season_days[season]][..., None, None], mode = 'full')
                #switch the tail (that goes into the following months) to follow the GF of the next month 
                if idx == 0 or idx == 1 or idx == 2:
                    idx_next = idx + 1
                elif idx == 3:
                    idx_next = 0

                tail_switch = signal.convolve(G_ds.sel(run = country_emit_dict[country_emit][idx_next]).fillna(0), 
                                     E_CO2_all_opts[unique_id][n:n+season_days[season]][..., None, None], mode = 'full')

                C_dict[season][season_days[season]:tail_switch[season_days[season]:C_dict[season].shape[0]].shape[0]+season_days[season]] = tail_switch[season_days[season]:C_dict[season].shape[0]]

                n = n + season_days[season]

            C = {}

            C['DJF'] = sparse.COO.from_numpy(np.pad(C_dict['DJF'],((((0),
                                               (365 - len(C_dict['DJF'])))),
                                             (0,0),(0,0))))
            C['MAM'] = sparse.COO.from_numpy(np.pad(C_dict['MAM'],((((season_days['DJF']),
                                               (365 - season_days['DJF'] - len(C_dict['MAM'])))),
                                             (0,0),(0,0))))
            C['JJA'] = sparse.COO.from_numpy(np.pad(C_dict['JJA'],((((season_days['DJF'] + season_days['MAM'] ),
                                               (365 - season_days['DJF'] - season_days['MAM'] - len(C_dict['JJA'])))),
                                             (0,0),(0,0))))
            C['SON'] = sparse.COO.from_numpy(np.pad(C_dict['SON'][:season_days['SON']], ((((season_days['DJF'] + season_days['MAM'] + season_days['JJA']),
                                                (365 - season_days['DJF'] - season_days['MAM'] - season_days['JJA'] - len(C_dict['SON'][:season_days['SON']])))),
                                              (0,0),(0,0))))
            C_sum = C['DJF']+C['MAM']+C['JJA']+C['SON']

            C_dense = sparse.COO.todense(C_sum)
            
            C_out = np_to_xr_time_specific(C_dense, G_ds, E_CO2_all_opts[unique_id], time_init = np.unique([i for i, x in enumerate(E_CO2_all_opts[unique_id]>0) if x])[0])
       
            ### country level impacts ###
            mask = country_mask.mask(C_out, lon_name = 'lon', lat_name = 'lat')
            for country_impacted in countries:
                if country_impacted == 'East Timor':
                    c_imp = 'Timor-Leste'
                elif country_impacted == 'Solomon Islands':
                    c_imp = 'Solomon Is.'
                else:
                    c_imp = country_impacted
                contiguous_mask = ~np.isnan(mask)& (mask == country_mask.map_keys(c_imp))
                country_impacted_ds = C_out.where(contiguous_mask)
                country_impacted_ds = country_impacted_ds.to_dataset(name = 'BC_conc')
                country_area = ds_area['area'].where(contiguous_mask)
                
                if nm == 'surface':
                    ## save out the concentrations and population weighted concentrations ##
                    conc_mean_out = ((country_impacted_ds['BC_conc']*country_area).sum(dim = ['lat','lon'])/country_area.sum()).mean(dim = ['s']).values #over one year
                    pop_weight_conc = country_impacted_ds['BC_conc'].weighted(regrid_area_ds['regrid_pop_count']).mean(dim = ['lat','lon','s']).values
                    
                    data.loc[country_impacted, 'BC_surface_mean_conc'] = conc_mean_out
                    data.loc[country_impacted, 'BC_pop_weight_mean_conc'] = pop_weight_conc

                elif nm == 'column':
                    conc_mean_out = ((country_impacted_ds['BC_conc']*country_area).sum(dim = ['lat','lon'])/country_area.sum()).mean(dim = ['s']).values #over one year
                    data.loc[country_impacted, 'BC_column_mean_conc'] = conc_mean_out

                print(data)
    data.to_csv(f'Outputs/individual_shutdowns/final_out_convolution/C_out_{country_emit}_{unique_id}_uniqueid_single_yr.nc')
    print(f'saved out {country_emit}, {unique_id} unique id, single year')
    print(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)
    lst = [data]
    del C_sum
    del data
    del lst
    gc.collect()
    print(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)



16
[0. 0. 0. ... 0. 0. 0.]
32 year range
                   BC_surface_mean_conc BC_pop_weight_mean_conc  \
China             4.234731322815064e-10   8.518790707186426e-10   
Australia                           NaN                     NaN   
India                               NaN                     NaN   
Myanmar                             NaN                     NaN   
Cambodia                            NaN                     NaN   
Laos                                NaN                     NaN   
Philippines                         NaN                     NaN   
Nepal                               NaN                     NaN   
Bangladesh                          NaN                     NaN   
Thailand                            NaN                     NaN   
Bhutan                              NaN                     NaN   
Brunei                              NaN                     NaN   
Singapore                           NaN                     NaN   
Papua New Guinea     

NameError: name 'C_init' is not defined

In [ ]:
# REMOVE REMOVE REMOVE ######## Set health data #########

# RR = 1.02 #global mean
# del_x = 10 #ug/m3
# beta = np.log(RR)/del_x


# #2019 mortalities to match 2019 population data from the GBD #20+, all gender mortalities https://vizhub.healthdata.org/gbd-results/
# I_val = {}
# I_val['China'] = 10462043.68
# I_val['Australia'] = 169053.20
# I_val['Cambodia'] = 96284.85
# I_val['Myanmar'] = 368031.84
# I_val['Laos'] = 35874.09
# I_val['Philippines'] = 557809.29
# I_val['Nepal'] = 170032.44
# I_val['Bangladesh'] = 740684.73
# I_val['Thailand'] = 486556.52
# I_val['Bhutan'] = 3713.40
# I_val['East Timor'] = 6240.29
# I_val['Solomon Islands'] = 5484.73
# I_val['Brunei'] = 1790.80
# I_val['Papua New Guinea'] = 49174.72
# I_val['India'] = 8288847.22
# I_val['Singapore'] = 23028.19
# I_val['Taiwan'] = 183845.08


# I_val_df = pd.DataFrame(I_val.values(), index = I_val.keys()).rename(columns = {0:'Ival'}) 
# I_val_df.index.rename('country', inplace = True)
# pop_df = country_df.loc[country_df['country'].isin(countries)].loc[country_df['POP_YEAR'] == 2019][['country','POP_EST']].set_index('country').groupby('country').max() #select 2019 population data
# I0_pop_df = pd.merge(pop_df, I_val_df, left_index=True, right_index=True) #combine Initial mortality and total population by country
# I0_pop_df['I_obs'] = I0_pop_df['Ival']/I0_pop_df['POP_EST'] #calculate initial mortality rate, I_obs

## END REMOVE ##

In [18]:
data = pd.DataFrame(columns = ['BC_surface_mean_conc','BC_pop_weight_mean_conc', 'BC_columnn_mean_conc'], index = countries) #create a dataframe where we output the mean concentration over the year + population weighted mean concentration (all at the surface)


In [24]:
data.loc['China', 'BC_surface_mean_conc'] = 2

In [25]:
data

,BC_surface_mean_conc,BC_pop_weight_mean_conc,BC_columnn_mean_conc
China,2,2,5
Australia,NaN,NaN,NaN
India,NaN,NaN,NaN
Myanmar,NaN,NaN,NaN
Cambodia,NaN,NaN,NaN
Laos,NaN,NaN,NaN
Philippines,NaN,NaN,NaN
Nepal,NaN,NaN,NaN
Bangladesh,NaN,NaN,NaN
Thailand,NaN,NaN,NaN


In [17]:
zipped = zip(['brian', 'dog'], ['katie', 'poodle'], ['lev0', 'cat'])
for i, x, y in zipped:
    print( i, x, y )

brian katie lev0
dog poodle cat


In [7]:
{utils.geos_chem_data_path}

{'/net/fs11/d0/emfreese/GCrundirs/IRF_runs/'}

In [ ]:
xr.open_mfdataset(f'/net/fs11/d0/emfreese/GCrundirs/IRF_runs/stretch_base/template/Jan/OutputDir/GEOSChem.AerosolMass.2016*', combine = 'by_coords', engine = 'netcdf4')

In [3]:
xr.open_dataset('/net/fs11/d0/emfreese/GCrundirs/IRF_runs/stretch_2x/Cambod/OutputDir/GEOSChem.AerosolMass.20160102_0000z.nc4')

<xarray.Dataset>
Dimensions:       (Xdim: 90, Ydim: 90, nf: 6, ncontact: 4, YCdim: 91,
                   XCdim: 91, lev: 72, time: 1)
Coordinates:
  * Xdim          (Xdim) float64 152.1 151.6 151.1 150.6 ... 74.88 74.38 73.9
  * Ydim          (Ydim) float64 -12.64 -13.11 -13.59 ... -86.79 -87.91 -88.98
  * nf            (nf) int32 1 2 3 4 5 6
  * ncontact      (ncontact) int32 1 2 3 4
    lons          (nf, Ydim, Xdim) float32 ...
    lats          (nf, Ydim, Xdim) float32 ...
  * lev           (lev) float64 1.0 2.0 3.0 4.0 5.0 ... 68.0 69.0 70.0 71.0 72.0
  * time          (time) datetime64[ns] 2016-01-02
Dimensions without coordinates: YCdim, XCdim
Data variables:
    cubed_sphere  |S1 ...
    contacts      (nf, ncontact) int32 ...
    orientation   (nf, ncontact) |S5 ...
    anchor        (nf, ncontact, ncontact) int32 ...
    corner_lons   (nf, YCdim, XCdim) float32 ...
    corner_lats   (nf, YCdim, XCdim) float32 ...
    AerMassBC     (time, lev, nf, Ydim, Xdim) float32 ...
    PM25          (time, lev, nf, Ydim, Xdim) float32 ...
    TotalOA       (time, lev, nf, Ydim, Xdim) float32 ...
Attributes:
    additional_vars:      contacts,orientation,anchor
    file_format_version:  2.90
    grid_mapping_name:    gnomonic cubed-sphere
    gridspec_file:        C90_gridspec.nc4
    stretch_factor:       2.0
    target_lat:           10.999999
    target_lon:           113.0